In [ ]:
from scholarly import scholarly
from ratelimit import limits, sleep_and_retry
from random import randint
from time import sleep
import time

from tqdm import tqdm

tqdm.pandas()

import pandas as pd
import xmltodict
from tqdm.notebook import tqdm, trange

### Read in URLs

In [ ]:
# loading 25494 unique OJS journal URLs
df = pd.read_csv("../data/gscholar_urls.csv")
df

In [ ]:
# sample for testing
# df = df.sample(10)
# df

### Search GScholar

In [ ]:
# test single query
scholarly.search_pubs(
    "site:periodicos.unipampa.edu.br/index.php/siepe"
)._get_total_results()

In [ ]:
# function to query gscholar max 60 times per hour with random sleep intervals

@sleep_and_retry
@limits(calls=60, period=3600)
def is_journal_present(url):
    try:
        sleep(randint(10, 50))
        return scholarly.search_pubs("site:" + url)._get_total_results()
    except Exception as e:
        print(repr(e))
        return repr(e)

In [ ]:
df["n_articles"] = df.progress_apply(lambda row: is_journal_present(row["url"]), axis=1)

In [ ]:
df

In [ ]:
# total OJS articles on gscholar
df["n_articles"].sum()

In [ ]:
# output to csv
df.to_csv("gscholar_urls_mapped.csv", encoding="utf-8", index=False)